# `Naive RAG`, `Advanced RAG` 구현 코드

### 실습 목표
Naive RAG와 Re-ranker를 도입한 Advanced RAG를 구축할 수 있습니다. 두 파이프라인의 응답 결과를 직관적으로 비교하며, 정량적 평가의 필요성을 체감합니다.

## 1. 환경 설정
필요한 라이브러리를 설치하고 불러옵니다.

In [ ]:
import os
import requests

# LangChain 관련 라이브러리
# PDF 로더를 위해 PyPDFLoader를 추가합니다.
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.chains.combine_documents import create_stuff_documents_chain

In [ ]:
import os
from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

OpenAI API Key: ········


## 1. 샘플 문서 로드
비교를 위해 의미 있는 내용이 충분히 담긴 문서를 로드합니다.  
여기서는 **한국지능정보사회진흥원**의 AI국가전략 보고서 중 "전국민 AI 일상화 실행 계획"을 사용하겠습니다.
 - https://www.nia.or.kr/site/nia_kor/ex/bbs/View.do?cbIdx=99952&bcIdx=27378&parentSeq=27378

In [5]:
loader = PyPDFLoader("./National_AI_Plan.pdf")
documents = loader.load()

## 문서를 잘 읽고 있는지 페이지를 달리하면서, 확인해보세요
print(documents[15].page_content[:500])

-6-
 Ⅲ. 국내 현황 진단◈전 세계적 디지털 모범국가로 나아가기 위한 정부의 전략 추진과민간의 선제적 혁신으로 전국민 AI일상화초석 마련○(정부)디지털 심화 시대에 대응한 신속한 전략･
정책추진을 통해 AI기술･
산업 경쟁력을 강화하고 민간 혁신을 뒷받침  * 대한민국 디지털 전략(‘22.9) → AI일상화·산업 고도화계획(’23.1)→초거대 AI경쟁력강화방안(‘23.4)-｢
AI법제정비단(’23.8~)｣
를 통해 AI확산에 따른 사회적 이슈를 정비하고,AI거버넌스 정립과 공동 번영을 위한 글로벌 협력 본격화○(민간)독자적 초거대 AI개발･
출시 본격화,중소･
스타트업 중심의다양한 응용서비스출시 등 산업 생태계 조성시작네이버, ‘클로바’LG, ‘엑사원’카카오, KoGPT SKT, ‘에이닷’KT, ‘믿음’
-의료 등 다양한 도메인에서의중소·스타트업 역시 초거대 AI기반전문서비스를 출시,체감되는 AI활용·확산을 주도
* 의료 AI 기업 ‘루닛’이 美정부의 암정복 프로젝트 ‘캔서 문샷


In [6]:
# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)

print(f"총 문서 개수: {len(docs)}")

총 문서 개수: 46


## 2. 공통 컴포넌트 준비
Naive RAG와 Advanced RAG에서 공통으로 사용할 LLM, 임베딩 모델, 프롬프트 템플릿을 정의합니다.

In [7]:
# LLM 초기화
llm = ChatOpenAI(
    model_name = "gpt-4o-mini",
    temperature=0
)

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model = "text-embedding-3-small",
)

In [9]:
prompt_template = """
당신은 주어진 문맥(context)만을 사용하여 사용자의 질문에 답변하는 AI 어시스턴트입니다.
문맥은 '전국민 AI 일상화 실행 계획' 보고서의 일부입니다.
주어진 문맥의 내용을 충실하게 요약하여, 질문에 명확하고 간결하게 답변해주세요.
절대로 문맥에 없는 내용을 지어내서 답변하면 안 됩니다. 문맥에서 답을 찾을 수 없다면 "문맥에서 관련 정보를 찾을 수 없습니다."라고 답변하세요.

[문맥]
{context}

[질문]
{question}

[답변]
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

Test_query = "정보통신전략위원회는 어떤 역할을 하나요?"

print("--- 공통 컴포넌트(LLM, Embedding, Prompt, Test_query) 준비 완료 ---")

--- 공통 컴포넌트(LLM, Embedding, Prompt, Test_query) 준비 완료 ---


## [TODO] 3. 기본 RAG 구현 (Naive RAG)
가장 기본적인 RAG 체인을 구현합니다. `검색(Retrieve) -> 생성(Generate)`의 단순한 구조입니다.  

빈 칸을 채워주세요

In [20]:
from operator import itemgetter

# FAISS 벡터 스토어 생성
naive_vectorstore = FAISS.from_documents(docs, embeddings)

# 기본 검색기 생성 (k=10)
# k=10은 10개의 가장 유사한 문서를 반환하도록 설정합니다.
# 즉, naive_vectorstore 를 runnable 하게 만들기 위해 as_retriever 를 호출
naive_retriever = naive_vectorstore.as_retriever(search_kwargs={"k": 10})

# Document Chain 생성
naive_chain = create_stuff_documents_chain(llm, prompt)

# Naive RAG 체인 구성 (파이프라인 방식)
# below code is made by GEMINI
# naive_rag_chain = (
#     RunnablePassthrough.assign(context=(lambda x: x["question"]) | naive_retriever)
#     | naive_chain
# )
# below code is made by 강사님
# 강사님 스타일 코드로 수정하고 TypeError를 해결합니다.
naive_rag_chain = (
    {"context": itemgetter("question") | naive_retriever, "question": RunnablePassthrough()}
    | naive_chain
)


def naive_rag(query):
    """기본 RAG 체인으로 답변을 생성합니다."""
    result = naive_rag_chain.invoke({"question": query})
    return result

# 테스트 쿼리 실행
print("\n--- Naive RAG 실행 결과 ---")
answer = naive_rag(Test_query)
print(f"질문: {Test_query}")
print(f"답변: {answer}")


--- Naive RAG 실행 결과 ---
질문: 정보통신전략위원회는 어떤 역할을 하나요?
답변: 정보통신전략위원회는 '전국민 AI 일상화 실행 계획'의 추진실적을 점검하는 역할을 합니다.


테스트 쿼리를 실행해 Naive RAG의 작동을 확인합니다.

In [11]:
# Naive RAG 실행
print("Test_query: ", Test_query)
print("="*75)
print("Naive RAG 답변:")
print(naive_rag(Test_query))

Test_query:  정보통신전략위원회는 어떤 역할을 하나요?
Naive RAG 답변:
정보통신전략위원회는 '전국민 AI 일상화 실행 계획'의 추진실적을 점검하는 역할을 합니다.


## [TODO] 4. Advanced RAG 구현
Reranking은 초기 검색 결과를 더 정교한 방법으로 재평가하여 문서 관련성을 재정렬하는 기법입니다. 여기서는 BERT 기반 Cross-Encoder를 활용한 Reranker를 구현합니다.

In [15]:
# Reranking은 초기 검색 결과를 더 정교한 방법으로 재평가하여 문서 관련성을 재정렬하는 기법입니다. 여기서는 BERT 기반 Cross-Encoder를 활용한 Reranker를 구현합니다.

# FAISS 벡터 스토어 생성
advanced_vectorstore = FAISS.from_documents(docs, embeddings)

# Document Chain 생성
document_chain = create_stuff_documents_chain(llm, prompt)

# Cross-Encoder Reranker 구현
cross_encoder = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
reranker = CrossEncoderReranker(model=cross_encoder, top_n=5)

# Reranking 검색기 생성
reranking_retriever = ContextualCompressionRetriever(
    base_retriever=advanced_vectorstore.as_retriever(search_kwargs={"k": 10}),
    base_compressor=reranker
)

# Reranking RAG 체인 구성
reranking_rag_chain = (
    {"context": reranking_retriever, "question": RunnablePassthrough()}
    | document_chain
)


def reranking_rag(query):
    """Reranking을 적용한 RAG 체인으로 답변을 생성합니다."""
    result = reranking_rag_chain.invoke(query)
    return result

# 테스트 쿼리 실행
print("\n--- Reranking RAG 실행 결과 ---")
reranking_answer = reranking_rag(Test_query)
print(f"질문: {Test_query}")
print(f"답변: {reranking_answer}")


--- Reranking RAG 실행 결과 ---
질문: 정보통신전략위원회는 어떤 역할을 하나요?
답변: 정보통신전략위원회는 '전국민 AI 일상화 실행 계획'의 추진실적을 점검하는 역할을 합니다.


테스트 쿼리를 실행해 Advanced RAG의 작동을 확인합니다.

In [13]:
# 테스트 쿼리로 문서 검색
retrieved_docs = advanced_vectorstore.similarity_search(Test_query, k=10)

# Cross-Encoder 점수 계산
def get_cross_encoder_scores(query, docs):
    """Cross-Encoder를 사용해 문서의 관련성 점수를 계산합니다."""
    text_pairs = [(query, doc.page_content) for doc in docs]
    scores = cross_encoder.score(text_pairs)
    return list(zip(docs, scores))

# 원본 검색 결과와 Reranker 적용 결과 비교
reranked_docs = reranker.compress_documents(retrieved_docs, Test_query)

print("원본 검색 결과:")
for i, doc in enumerate(retrieved_docs):
    print(f"\n문서 {i+1}: {doc.page_content[:150]}...")

docs_with_scores = get_cross_encoder_scores(Test_query, retrieved_docs)
docs_with_scores.sort(key=lambda x: x[1], reverse=True)

print("="*75)

print("\nCross-Encoder 점수로 재정렬된 결과 (상위 5개):")
for i, (doc, score) in enumerate(docs_with_scores[:5]):
    print(f"\n문서 {i+1} (점수: {score:.4f}): {doc.page_content[:150]}...")

원본 검색 결과:

문서 1: -v-
□ (공공서비스) 수입식품 위험성을 예측･
분석하는 AI솔루션을 적용(식약처)하고,수돗물 수질관리등을 최적화하는 ‘AI정수장’구축 검토(환경부)○강수･
폭염･
강풍 예보지원AI를 통해신속･
정확한기상정보를 제공(기상청),구인정보,직무데이터 등을 분석하여 구직자가...

문서 2: -27-
 Ⅵ. 추진체계 및 계획□ 추진체계 및 향후계획ㅇ ’정보통신전략위원회‘를 통해 동 실행계획의 추진실적 점검 ㅇ 각 부처 소관 영역의 AI일상화 신규과제 발굴,예산 반영 협의ㅇ 개발･
실증 및 현장적용 단계에서 효과가 검증된 AI제품･
서비스에대해서는 민관 협업...

문서 3: -20-
 3. AI를 가장 잘 사용하는 똑똑한 정부를 만들겠습니다 <3-1> 고품질 대국민 공공서비스 제공◇ AI기반 정수 처리, 식품 안전, 기상 예보 및 지능형 민원대응 등을 통한 대국민 공공서비스 품질의 고도화 추진□ 건강한 식품･식수 환경 제공 ○(수입식품 검...

문서 4: -30-
 ④ AI 일상화 기반을 선제적으로 조성하겠습니다추진과제추진시기소관부처▶ AI 문해력 제고AI 리터러시 함양 교수학습자료 개발 및 인정과목 개설’23교육부과기정통부디지털 튜터 배치 및 디지털 문제해결센터 운영’23교육부SW 미래채움센터 운영(’23년 13...

문서 5: -23-
 <3-3> 행정기관 내부업무 효율화◇ 특허, 산림보호, 통관, 통계 등 행정영역 전반에 AI 활용을 확대하여, 행정 업무 효율성 제고 및 국민 체감 극대화○(특허) 특허문서 등을 학습한 특허전용 초거대 AI언어모델을 구축하고,이를 특허문서 검색･
분류 등 심...

문서 6: -26-
 <4-2> AI 윤리･신뢰성 강화◇ 최근 전 세계적으로 강조되는 AI 윤리·신뢰성 이슈에 대응하여, 국내기업이 정부사업 등 AI 개발·적용 과정에서 윤리·신뢰성을 확보하도록 지원□(민관협업 신뢰성 확립)신뢰할 수 있는 AI개발안내서(’22)를 기반으로 AI제...

문서 7: -29-
 ③ AI를 가장 잘 

In [14]:
# Reranking RAG 실행
print("Test_query: ", Test_query)
print("="*75)
print("\nReranking RAG 답변:")
print(reranking_rag(Test_query))

Test_query:  정보통신전략위원회는 어떤 역할을 하나요?

Reranking RAG 답변:
정보통신전략위원회는 '전국민 AI 일상화 실행 계획'의 추진실적을 점검하는 역할을 합니다.


## 5. Naive RAG vs Advanced RAG 실행 및 비교
동일한 질문에 대해 두 RAG 파이프라인이 어떻게 다른 답변을 생성하는지 비교해봅니다. 보고서의 핵심 내용을 잘 요약하는지 확인해보세요.

In [21]:
query = "보고서에 언급된 '전국민 AI 일상화'를 위한 3대 추진 전략과 각각의 핵심 과제는 무엇인가요?"

In [ ]:
print(f"질문 : {query}")
print("=" * 50)

# Naive RAG 실행
print("Naive RAG 답변 생성!")
naive_answer = naive_rag(query)
print(f"\n Naive RAG 답변 : \n{naive_answer}")
print("=" * 50)

# Reranking RAG 실행
print("Reranking RAG 답변 생성!")
reranking_answer = reranking_rag(query)
print(f"\n Naive RAG 답변 : \n{reranking_answer}")

질문 : 보고서에 언급된 '전국민 AI 일상화'를 위한 3대 추진 전략과 각각의 핵심 과제는 무엇인가요?
Naive RAG 답변 생성!

 Naive RAG 답변 : 
보고서에 언급된 '전국민 AI 일상화'를 위한 3대 추진 전략과 각각의 핵심 과제는 다음과 같습니다:

1. **AI로 국민 일상을 풍요롭게 하겠습니다**
   - 사회적 약자 돌봄 및 배려 서비스 제공
   - 의료 및 보건 서비스 품질 제고
   - 아동 및 청소년 성장 환경 개선

2. **AI 내재화로 산업･일터를 혁신하겠습니다**
   - 제조 및 서비스업에 AI 적용
   - 품질 개선 및 생산성 향상
   - 공공행정에 AI 도입

3. **AI를 가장 잘 사용하는 똑똑한 정부를 만들겠습니다**
   - 행정업무 효율성 제고
   - 데이터･AI 중심의 디지털 플랫폼 정부 실현
   - AI 기반의 공공 서비스 개선

이러한 전략을 통해 AI를 국민의 삶 전반에 전방위적으로 확산하고, 디지털 모범국가로 도약하는 것을 목표로 하고 있습니다.
Reranking RAG 답변 생성!


# 평가 자동화 프레임워크 : RAGAS

### 실습 목표
구축한 Naive RAG와 Advanced RAG 시스템의 성능을 RAGAS 프레임워크를 사용하여 정량적으로 측정하고, 객관적인 지표로 비교 분석하는 능력을 기릅니다.

https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/


## 1. 환경 설정
필요한 라이브러리를 불러옵니다.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

## 2. RAGAS 평가를 위한 데이터셋 준비
- RAGAS 평가에 사용할 질문-정답 쌍을 정의합니다.
- 이 질문과 정답은 '전국민 AI 일상화 실행 계획' 보고서 내용을 기반으로 직접 작성해야 합니다.
- 최소 5~10개 이상의 질문-정답 쌍을 만드는 것을 권장합니다.

In [ ]:
eval_data = {
    "question": [
        "전국민 AI 일상화 실행 계획의 주요 목표는 무엇인가요?",
        "AI 일상화 계획에서 초거대 AI 확산 지원을 위해 어떤 방안을 추진하고 있나요?",
        "AI 친화적인 디지털 교육 환경을 구축하기 위한 계획은 무엇인가요?",
        "시니어 및 소상공인을 위한 AI 활용 지원 방안에는 어떤 것들이 포함되나요?",
        "AI 기술을 활용한 사회 문제 해결 분야의 주요 과제는 무엇인가요?"
        # 여기에 보고서 내용을 기반으로 더 많은 질문을 추가하세요.
        # 예: "AI 서비스 활용 역량을 강화하기 위한 국민 참여 프로그램에는 어떤 것이 있나요?"
        # 예: "초거대 AI 기술을 공공 서비스에 적용하는 사례는 무엇인가요?"
    ],
    "ground_truth": [
        "전국민 AI 일상화 실행 계획의 주요 목표는 누구나 일상 속에서 AI 혜택을 누리는 'AI 일상화 시대'를 구현하고, AI를 활용하여 국가 전반의 생산성을 높이는 것입니다.",
        "초거대 AI 확산 지원을 위해 AI 인프라 확충, 초거대 AI 기반 서비스 개발 지원, AI 전문기업 육성 등을 추진하고 있습니다.",
        "AI 친화적인 디지털 교육 환경 구축을 위해 유아부터 성인까지 전 국민 대상의 AI 교육 기회를 확대하고, 교육 콘텐츠 및 플랫폼을 개발하며, AI 전문 강사를 양성할 계획입니다.",
        "시니어 및 소상공인을 위한 AI 활용 지원 방안에는 AI 기반 돌봄 서비스 제공, 소상공인 AI 비서 및 챗봇 도입 지원, AI 기반 생산성 향상 도구 보급 등이 포함됩니다.",
        "AI 기술을 활용한 사회 문제 해결 분야의 주요 과제는 국민 안전 및 건강 증진, 사회적 약자 지원, 환경 문제 해결 등 다양한 분야에서 AI 기반 솔루션을 도입하고 확산하는 것입니다."
        # 여기에 각 질문에 대한 정확한 '정답'을 직접 작성하세요.
    ]
}
eval_df = pd.DataFrame(eval_data)

## Naive RAG 시스템 평가
- RAGAS 평가에 필요한 'contexts' 및 'answer' 필드 채우기
- 각 질문에 대해 Naive RAG 시스템을 실행하고 결과 저장

In [ ]:
print("--- Naive RAG 시스템 평가 시작 ---")

naive_results = []
for i, row in eval_df.iterrows():
    question = row["question"]
    # Naive RAG의 답변 생성
    answer = naive_rag(question)

    # Naive RAG의 컨텍스트 검색
    retrieved_docs = naive_retriever.invoke(question)
    contexts = [doc.page_content for doc in retrieved_docs]

    naive_results.append({
        "question": question,
        "answer": answer,
        "contexts": contexts,
        "ground_truth": row["ground_truth"]
    })

naive_dataset = Dataset.from_list(naive_results)

# Naive RAG 평가 실행
naive_metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision
]

naive_eval_result = evaluate(
    dataset=naive_dataset,
    metrics=naive_metrics,
    llm=llm, # RAGAS 평가를 위한 LLM
    embeddings=embeddings, # RAGAS 평가를 위한 임베딩 모델
)

print("\n--- Naive RAG 평가 결과 ---")
print(naive_eval_result)
print(naive_eval_result.to_pandas())

## Advanced RAG 시스템 평가
- RAGAS 평가에 필요한 'contexts' 및 'answer' 필드 채우기
- 각 질문에 대해 Advanced RAG 시스템을 실행하고 결과 저장

In [ ]:
print("\n--- Advanced RAG 시스템 평가 시작 ---")

advanced_results = []
for i, row in eval_df.iterrows():
    question = row["question"]
    # Advanced RAG의 답변 생성
    answer = reranking_rag(question)

    # Advanced RAG의 컨텍스트 검색 (reranking_retriever는 압축된 컨텍스트를 반환)
    # RAGAS는 원본 컨텍스트를 필요로 할 수 있으므로, reranking_retriever가 반환하는 내용을 확인하세요.
    # 여기서는 reranking_retriever가 최종적으로 선택한 문서를 반환한다고 가정합니다.
    retrieved_docs_advanced = reranking_retriever.invoke(question)
    contexts_advanced = [doc.page_content for doc in retrieved_docs_advanced]

    advanced_results.append({
        "question": question,
        "answer": answer,
        "contexts": contexts_advanced,
        "ground_truth": row["ground_truth"]
    })

advanced_dataset = Dataset.from_list(advanced_results)

# Advanced RAG 평가 실행
advanced_metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision
]

advanced_eval_result = evaluate(
    dataset=advanced_dataset,
    metrics=advanced_metrics,
    llm=llm, # RAGAS 평가를 위한 LLM
    embeddings=embeddings, # RAGAS 평가를 위한 임베딩 모델
    # is_html_report=True # HTML 리포트 생성
)

print("\n--- Advanced RAG 평가 결과 ---")
print(advanced_eval_result)
print(advanced_eval_result.to_pandas())

## 평가 결과 비교

In [ ]:
print("\n--- RAG 시스템 성능 비교 ---")
naive_df = naive_eval_result.to_pandas()
advanced_df = advanced_eval_result.to_pandas()

comparison_df = pd.DataFrame({
    "Metric": ["faithfulness", "answer_relevancy", "context_recall", "context_precision"],
    "Naive RAG": [
        naive_df['faithfulness'].mean(),
        naive_df['answer_relevancy'].mean(),
        naive_df['context_recall'].mean(),
        naive_df['context_precision'].mean()
    ],
    "Advanced RAG": [
        advanced_df['faithfulness'].mean(),
        advanced_df['answer_relevancy'].mean(),
        advanced_df['context_recall'].mean(),
        advanced_df['context_precision'].mean()
    ]
})

print(comparison_df)

In [ ]:
comparison_df.set_index("Metric").plot(kind='bar', figsize=(10, 6))
plt.title("RAG System Performance Comparison")
plt.ylabel("Score")
plt.ylim(0, 1)
plt.xticks(rotation=45, ha='right')
plt.legend(title="RAG System")
plt.tight_layout()
plt.show()